In [1]:
### Script to collect data from WSDC Points Registry API
# by Sarah Rempel, API posts were from Connie Wang's Jan 2018 code:
#       https://github.com/conniewang3/WSDC-Project/tree/master/part1
# 2019-12-28, updated to collect names on 2021-02-27, updated to Jupyter Notebooks 2023-02-02

In [2]:
import pandas as pd
import openpyxl, requests, pprint, time
from datetime import date
import simplejson as json

In [3]:
### Install packages
## with conda:
#import sys
#!conda install --yes --prefix {sys.prefix} simplejson
## with pip:
#import sys
#!{sys.executable} -m pip install numpy

In [4]:
API_url = 'https://points.worldsdc.com/lookup/find'

point_df = pd.DataFrame(columns = ['wsdc_id', 'level_points', 'allowed_level', 'required_level', \
                                   'event_level', 'event_name', 'event_location', 'event_date', \
                                   'points', 'result', 'role', 'first_name', 'last_name'])
today = date.today() # the collection date will be appended to the excel file so I know when I ran it


In [ ]:
# Pick the start and end numbers. Note that it's best to do 5-10k numbers at a time (takes ~1-2hr to run)
#    instead of doing the whole thing at once (which takes 4-6 hours)
start = 13758 # My (JD's) number is 13758 to start and edit at 13759 to not include anyone else.
end = 13759
# up to 21606 as of 2023-04-01
print(point_df)

Empty DataFrame
Columns: [wsdc_id, level_points, allowed_level, required_level, event_level, event_name, event_location, event_date, points, result, role, first_name, last_name]
Index: []


In [6]:
# Loop to go through every WSDC number
for wsdc_id in range(start, end):
    try:
        response = requests.post(API_url, {'q': wsdc_id}).json()
        
        print(response)

        # only cases with valid WSDC IDs containing WCS placements
        if len(response) > 2 and response['placements'] != [] and 'West Coast Swing' in response['placements'].keys():
            # print('Westie confirmed.')
            westie = response['placements']['West Coast Swing']

            #Extract dancer's level information and name
            allowed_level = response['level']['allowed']
            required_level = response['level']['required']
            first_name = response['dancer']['first_name']
            last_name = response['dancer']['last_name']
            
            #Extract information on a per level and per event basis
            for event_level in westie:
                # print(event_level + " of " + str(len(westie)))
                tot_points = westie[event_level]['total_points']
                eventList = westie[event_level]['competitions']

                for i in range(len(eventList)):
                    #print(str(i) + " of " + str(len(eventList)))
                    event_name = eventList[i]['event']['name']
                    event_date = eventList[i]['event']['date']
                    event_location = eventList[i]['event']['location']
                    points = eventList[i]['points']
                    result = eventList[i]['result']
                    role = eventList[i]['role']
                    #print(role)

                    obs = {'wsdc_id': wsdc_id, 'level_points': str(event_level)+'_'+str(tot_points), \
                           'allowed_level': allowed_level, 'required_level': required_level, \
                           'event_level': event_level, 'event_name': event_name, \
                           'event_location': event_location, 'event_date': event_date, \
                           'points': points, 'result': result, 'role': role, \
                           'first_name': first_name, 'last_name': last_name}
                    
                    point_df_new_row = pd.DataFrame(obs, index = [0])
                    point_df = pd.concat([point_df, point_df_new_row], ignore_index = True)   
                    
            if wsdc_id % 2000 == 0:
                point_df.to_csv('C:\\Users\\nafzi\\Desktop\\WSDCdata\\Point_DF_'+str(today)+'updating.csv')
            
        print('Dancer #'+str(wsdc_id)+' completed.')
        # sleep for 0.5 seconds on each for loop to let WSDC website rest
        time.sleep(0.5)
                    
                    #print('Westie #'+str(wsdc_id)+' got '+str(points)+' points as a '+role+' in '+event_date \
                    #     +'. They are level '+allowed_level+'.')
        #else:
               #print('No such westie with #' + str(wsdc_id) + ' exists.')
    except ValueError as IndexError:
        continue
print('That\'s all the westies!')


{'type': 'dancer', 'dancer': {'id': 13412, 'first_name': 'JD', 'last_name': 'Nafziger', 'wscid': 13758}, 'level': {'required': 'ADV', 'allowed': 'ADV', 'reason': ''}, 'placements': {'West Coast Swing': {'ADV': {'division': {'id': 6, 'name': 'Advanced', 'abbreviation': 'ADV'}, 'total_points': 27, 'competitions': [{'role': 'leader', 'points': 1, 'event': {'id': 259, 'name': 'Swingover', 'location': 'Orlando, FL', 'url': 'http://www.swingoverevent.com', 'date': 'April 2023'}, 'result': 'F'}, {'role': 'leader', 'points': 12, 'event': {'id': 18, 'name': 'Easter Swing', 'location': 'Bellevue, WA', 'url': 'http://www.easterswing.org/', 'date': 'April 2023'}, 'result': '2'}, {'role': 'leader', 'points': 2, 'event': {'id': 142, 'name': 'The Chicago Classic', 'location': 'Chicago, IL', 'url': 'http://www.thechicagoclassic.com', 'date': 'March 2023'}, 'result': '5'}, {'role': 'leader', 'points': 6, 'event': {'id': 125, 'name': 'Swing City Chicago', 'location': 'Chicago, IL', 'url': 'http://www.sw

In [7]:
# Export dataframe to a csv file with the path shown
# point_df.to_csv('C:\\Users\\nafzi\\Desktop\\WSDCdata\\Point_DF_'+str(start)+'-'+str(end)+'_'+str(today)+'.csv')

# response = requests.post(API_url, {'q': wsdc_id}).json()
#response is the following:
{'type': 'dancer', 
'dancer': {'id': 13412, 'first_name': 'JD', 'last_name': 'Nafziger', 'wscid': 13758}, 
'level': {'required': 'ADV', 'allowed': 'ADV', 'reason': ''}, 
'placements': {
    'West Coast Swing': 
    {'ADV': {'division': {'id': 6, 'name': 'Advanced', 'abbreviation': 'ADV'}, 'total_points': 27, 
        'competitions': 
            [{'role': 'leader', 'points': 1, 'event': {'id': 259, 'name': 'Swingover', 'location': 'Orlando, FL', 'url': 'http://www.swingoverevent.com', 'date': 'April 2023'}, 'result': 'F'}, 
            {'role': 'leader', 'points': 12, 'event': {'id': 18, 'name': 'Easter Swing', 'location': 'Bellevue, WA', 'url': 'http://www.easterswing.org/', 'date': 'April 2023'}, 'result': '2'}, 
            {'role': 'leader', 'points': 2, 'event': {'id': 142, 'name': 'The Chicago Classic', 'location': 'Chicago, IL', 'url': 'http://www.thechicagoclassic.com', 'date': 'March 2023'}, 'result': '5'}, 
            {'role': 'leader', 'points': 6, 'event': {'id': 125, 'name': 'Swing City Chicago', 'location': 'Chicago, IL', 'url': 'http://www.swingcitychicago.com', 'date': 'October 2022'}, 'result': '3'}, 
            {'role': 'leader', 'points': 6, 'event': {'id': 142, 'name': 'The Chicago Classic', 'location': 'Chicago, IL', 'url': 'http://www.thechicagoclassic.com', 'date': 'March 2022'}, 'result': '3'}]}, 
    'INT': {'division': {'id': 5, 'name': 'Intermediate', 'abbreviation': 'INT'}, 'total_points': 36, 
        'competitions': 
            [{'role': 'follower', 'points': 1, 'event': {'id': 259, 'name': 'Swingover', 'location': 'Orlando, FL', 'url': 'http://www.swingoverevent.com', 'date': 'April 2023'}, 'result': 'F'}, 
                {'role': 'follower', 'points': 2, 'event': {'id': 142, 'name': 'The Chicago Classic', 'location': 'Chicago, IL', 'url': 'http://www.thechicagoclassic.com', 'date': 'March 2022'}, 'result': '5'}, 
                {'role': 'follower', 'points': 1, 'event': {'id': 244, 'name': 'Rose City Swing', 'location': 'Portland, OR', 'url': 'http://www.rosecityswing.com', 'date': 'February 2022'}, 'result': 'F'}, 
                {'role': 'leader', 'points': 1, 'event': {'id': 211, 'name': 'Atlanta Swing Classic', 'location': 'Atlanta, GA', 'url': 'http://www.atlantaswingclassic.com', 'date': 'October 2021'}, 'result': 'F'}, 
                {'role': 'follower', 'points': 2, 'event': {'id': 87, 'name': 'C.A.S.H. Bash Weekend', 'location': 'Twinsburg, OH', 'url': 'http://www.cashdanceclub.org', 'date': 'December 2019'}, 'result': '5'}, 
                {'role': 'leader', 'points': 10, 'event': {'id': 125, 'name': 'Swing City Chicago', 'location': 'Chicago, IL', 'url': 'http://www.swingcitychicago.com/', 'date': 'October 2019'}, 'result': '1'}, 
                {'role': 'leader', 'points': 2, 'event': {'id': 131, 'name': 'Meet Me in St. Louis Swing Dance Championships', 'location': 'St. Louis, MO', 'url': 'http://www.MeetMeInStLouisSDC.com', 'date': 'September 2019'}, 'result': '5'}, 
                {'role': 'leader', 'points': 6, 'event': {'id': 131, 'name': 'Meet Me in St. Louis Swing Dance Championships', 'location': 'St. Louis, MO', 'url': 'http://www.meetmeinstlouissdc.com', 'date': 'September 2018'}, 'result': '3'}, 
                {'role': 'leader', 'points': 1, 'event': {'id': 76, 'name': 'Michigan Dance Classic', 'location': 'Dearborn, MI', 'url': '', 'date': 'June 2018'}, 'result': 'F'}, 
                {'role': 'leader', 'points': 10, 'event': {'id': 242, 'name': 'Derby City Swing', 'location': 'Louisville, KY', 'url': '', 'date': 'January 2018'}, 'result': '1'}]}, 
    'NOV': {'division': {'id': 4, 'name': 'Novice', 'abbreviation': 'NOV'}, 'total_points': 17, 
        'competitions': 
            [{'role': 'leader', 'points': 8, 'event': {'id': 76, 'name': 'Michigan Dance Classic', 'location': 'Dearborn, MI', 'url': '', 'date': 'June 2017'}, 'result': '2'}, 
                {'role': 'leader', 'points': 1, 'event': {'id': 61, 'name': 'SwingDiego', 'location': 'San Diego, CA', 'url': '', 'date': 'May 2017'}, 'result': 'F'}, 
                {'role': 'leader', 'points': 6, 'event': {'id': 134, 'name': 'Swing Dance America', 'location': 'Lake Geneva, IL', 'url': '', 'date': 'April 2017'}, 'result': '3'}, 
                {'role': 'leader', 'points': 1, 'event': {'id': 242, 'name': 'Derby City Swing', 'location': 'Louisville, KY', 'url': '', 'date': 'January 2017'}, 'result': 'F'}, 
                {'role': 'leader', 'points': 1, 'event': {'id': 125, 'name': 'Swing City Chicago', 'location': 'Chicago, IL', 'url': '', 'date': 'October 2016'}, 'result': 'F'}]}, 
    'NEW': {'division': {'id': 3, 'name': 'Newcomer', 'abbreviation': 'NEW'}, 'total_points': 6, 
        'competitions': 
            [{'role': 'leader', 'points': 6, 'event': {'id': 242, 'name': 'Derby City Swing', 'location': 'Louisville, KY', 'url': '', 'date': 'January 2016'}, 'result': '3'}]}
    }} #close placements #close "West Coast Swing"
} #close response

